In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data = []
labels = []

# Clean 폴더 이미지 불러오기
clean_dir = 'Data_PlasticCup/clean'
for filename in os.listdir(clean_dir):
    img = cv2.imread(os.path.join(clean_dir, filename))
    img = cv2.resize(img, (128, 128))
    data.append(img)
    labels.append(0)  # 0 = 깨끗한 컵

# NG 폴더 이미지 불러오기
ng_dir = 'Data_PlasticCup/ng'
for filename in os.listdir(ng_dir):
    img = cv2.imread(os.path.join(ng_dir, filename))
    img = cv2.resize(img, (128, 128))
    data.append(img)
    labels.append(1)  # 1 = 더러운 컵

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [3]:
# 모델 정의
model = Sequential()

# 컨볼루션 레이어 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 2 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 3
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))

# fully connected 레이어
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 데이터를 NumPy 배열로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=16)

# 모델 저장
model.save('plastic_cup_classifier.h5')

c:\Users\lavie\.conda\envs\ENV00\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.5417 - loss: 76.7256 - val_accuracy: 0.7872 - val_loss: 2.7668
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.6425 - loss: 2.6204 - val_accuracy: 0.7872 - val_loss: 0.3762
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.7238 - loss: 0.6136 - val_accuracy: 0.9787 - val_loss: 0.1216
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.8273 - loss: 0.4228 - val_accuracy: 0.9787 - val_loss: 0.3121
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9626 - loss: 0.1613 - val_accuracy: 1.0000 - val_loss: 9.2173e-04
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9566 - loss: 0.1994 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9688 - loss: 0.0640 - val_accuracy: 1.0000 - val_loss: 0.0118
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9784 - loss: 0.0390 - val_accuracy: 1.000

In [4]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('plastic_cup_classifier.h5')

# 테스트 데이터셋 준비
x_test = np.array(x_val)
y_test = np.array(y_val)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc * 100:.2f}%')

# 예측값 구하기
y_pred = (model.predict(x_test) > 0.5).astype(int)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (using sklearn.metrics): {accuracy * 100:.2f}%')

2/2 - 0s - 87ms/step - accuracy: 1.0000 - loss: 4.1434e-15
Test accuracy: 100.00%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Accuracy (using sklearn.metrics): 100.00%
